In [1]:
from rsome import ro
from rsome import grb_solver as grb
from math import sqrt
import pandas as pd
import numpy as np

## Sudoku

In [2]:
# Sudoku puzzle
# zeros represent unknown numbers
puzzle = np.array([[5, 3, 0, 0, 7, 0, 0, 0, 2],
                   [6, 0, 0, 1, 9, 5, 0, 0, 0],
                   [0, 9, 8, 0, 0, 0, 0, 6, 0],
                   [8, 0, 0, 0, 6, 0, 0, 0, 3],
                   [4, 0, 0, 8, 0, 3, 0, 0, 1],
                   [7, 0, 0, 0, 2, 0, 0, 0, 6],
                   [0, 6, 0, 0, 0, 0, 2, 8, 0],
                   [0, 0, 0, 4, 1, 9, 0, 0, 5],
                   [0, 0, 0, 0, 8, 0, 0, 7, 9]])

# create model and binary decision variables
model = ro.Model()
x = model.dvar((9, 9, 9), vtype='B')

# objective is set to be zero
model.min(0 * x.sum())

# constraints 1 to 3
model.st(x.sum(axis=0) == 1,
         x.sum(axis=1) == 1,
         x.sum(axis=2) == 1)

# constraints 4
i, j = np.where(puzzle)
model.st(x[i, j, puzzle[i, j]-1] == 1)

# constraints 5
for i in range(0, 9, 3):
    for j in range(0, 9, 3):
        model.st(x[i: i+3, j: j+3, :].sum(axis=(0, 1)) == 1)

# solve the integer programming problem
model.solve(grb)

Academic license - for non-commercial use only - expires 2022-07-30
Using license file C:\Users\kings\gurobi.lic
Being solved by Gurobi...
Solution status: 2
Running time: 0.0100s


In [3]:
x_sol = x.get().astype('int')   # retrieve the solution as an array of integers

print((x_sol * np.arange(1, 10).reshape((1, 1, 9))).sum(axis=2))

[[5 3 4 6 7 8 9 1 2]
 [6 7 2 1 9 5 3 4 8]
 [1 9 8 3 4 2 5 6 7]
 [8 5 9 7 6 1 4 2 3]
 [4 2 6 8 5 3 7 9 1]
 [7 1 3 9 2 4 8 5 6]
 [9 6 1 5 3 7 2 8 4]
 [2 8 7 4 1 9 6 3 5]
 [3 4 5 2 8 6 1 7 9]]


## Course Selection

Linda Lee, a second year BCG student, would like to determine her course schedule for her two semesters. Linda has created a list of twenty potential courses that most interest her, shown in table below. Linda has ranked her interest in each of these courses as a number between 3 and 5. The course data is extracted from http://web.nbs.ntu.edu.sg/undergrad/common/contents/courselist.asp 

 **Course Number** | **Course Code** | **Course Title**                                         | **Semester** | **Prerequisites** | **Interest Level** 
-------------------|-----------------|----------------------------------------------------------|---------------------|-------------------|--------------------
 1                 | AB0403          | Decision Making with Programming & Analytics             | 1,2                 |                   | 5                  
 2                 | BA2201          | Actuarial Economics                                      | 2                   |                   | 5                  
 3                 | BU5201          | Business Finance                                         | 1,2                 |                   | 4                  
 4                 | BE1401          | Business Operations & Process                            | 1,2                 |                   | 4                  
 5                 | BT2403          | Service Operations Management                            | 1                   |                   | 4                  
 6                 | BC3406          | Business Analytics Consulting                            | 2                   | 10, 13            | 3                  
 7                 | AB0602          | Communication Management Strategies                      | 1,2                 |                   | 3                  
 8                 | AB5103          | Audit Analytics                                          | 1,2                 | 1                 | 5                  
 9                 | BF3224          | Equity Investing with Big Data                           | 2                   | 13                | 4                  
 10                | BC2406          | Analytics I: Visual & Predictive Techniques              | 1                   | 1                 | 4                  
 11                | BC2407          | Analytics II: Visual & Predictive Techniques             | 2                   | 1,10              | 3                  
 12                | BM2507          | Marketing Analytics                                      | 1,2                 | 13                | 3                  
 13                | AB1202          | Statistics & Analysis                                    | 1,2                 |                   | 4                  
 14                | BC2408          | Supply Chain Analytics                                   | 2                   | 4                 | 4                  
 15                | BC2410          | Prescriptive Analytics: From Data to Decision            | 2                   | 1                 | 5                  
 16                | BU5644          | Smarter Brain? The Science of Decision Making Management | 2                   |                   | 5                  
 17                | AB0502          | Managing Sustainability                                  | 1                   |                   | 5                  
 18                | BM2501          | Market Behaviour                                         | 1,2                 | 20                | 3                  
 19                | BM2504          | Integrated Marketing Communications                      | 2                   | 20                | 3                  
 20                | AB1501          | Marketing                                                | 1,2                 |                   | 4                  


Linda is allowed to take at most five courses in each semester. In determining her course schedule, she needs to consider the following:

* Linda can only take a course if she has completed or is currently taking all courses that are prerequisites for the course. 
* In Semester 1, Linda must take at least three of the following five courses: course 1, course 3, course 4, course 5, and course 20.
* If Linda takes course 12, she will not be allowed to take course 18, because these two courses cover fairly similar materials.
* Linda would like to take at least two courses in Marketing (course 12, 18, and 20) and at least two courses in Analytics (course 1, 8, 10, 11, 14, 15).

In [4]:
## load the course information
course = pd.read_csv("Course_info.csv")
course

,Course Number,Course Code,Course Title,Semester,Prerequisites,Interest Level
0,1,AB0403,Decision Making with Programming & Analytics,"1,2",NaN,5
1,2,BA2201,Actuarial Economics,2,NaN,5
2,3,BU5201,Business Finance,"1,2",NaN,4
3,4,BE1401,Business Operations & Process,"1,2",NaN,4
4,5,BT2403,Service Operations Management,1,NaN,4
5,6,BC3406,Business Analytics Consulting,2,"10, 13",3
6,7,AB0602,Communication Management Strategies,"1,2",NaN,3
7,8,AB5103,Audit Analytics,"1,2",1,5
8,9,BF3224,Equity Investing with Big Data,2,13,4
9,10,BC2406,Analytics I: Visual & Predictive Techniques,1,1,4


In [5]:
#### process data
n = len(course['Semester'])
# assign values to aij
a = np.zeros((2,n))
for j in range( n ):
    if course['Semester'][j] == '1,2':
        a[0][j] = 1
        a[1][j] = 1
    elif course['Semester'][j] == '1':
        a[0][j] = 1
    else:
        a[1][j] = 1
I = np.array( course.iloc[:,5] )
print(a, '\n', I)

[[1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.]] 
 [5 5 4 4 4 3 3 5 4 4 3 3 4 4 5 5 5 3 3 4]


In [6]:
## build up the model
model = ro.Model("Course Selection")
x = model.dvar( (2,n), "B" )

**Objective:** Maximize the total interest level
$$
\max \sum_{j = 1}^n I_j (x_{1,j} + x_{2,j})
$$

In [7]:
model.max( x.sum( axis = 0) @ I )

**Constraints**
* A course is selected only if it is offered: 
$
x_{i,j} \leq a_{i,j} \quad \forall i \in \{1,2\}, j \in \{1,2,...,20\}
$

* Each course can be selected at most once: 
$
x_{1,j} + x_{2,j} \leq 1 \quad \forall j \in \{1,2,...,20\}
$

* Prerequisite constraints (two examples): 
$
x_{1,6} + x_{2,6} \leq (x_{1,10} + x_{2,10} + x_{1,13} + x_{2,13})/2 
$

If course 6, 10, 13 are available for both semester 1 and semester 2, then we need to make sure that course 10 and course 13 are selected before the choosing of course 6 in semester 1: 
$
x_{1,6} \leq (x_{1,10} + x_{1,13})/2 
$

Another example:
$
\sum_{i = 1}^2 x_{i,8} \leq \sum_{i = 1}^2 x_{i,1}
$

* Each semester, at most 5 courses can be selected: 
$
\sum_{j = 1}^{20} x_{i,j} \leq 5 \quad \forall i \in \{1,2\}
$

* At least 3 of the 5 courses must be selected in sem1: 
$
x_{1,1} + x_{1,3} + x_{1,4} + x_{1,5} + x_{1,20} \geq 3 
$

* If Linda takes course 12, she will not be allowed to take course 18: 
$
\sum_{i = 1}^2 x_{i,12} + \sum_{i = 1}^2 x_{i,18} <= 1
$

* At least two courses in Marketing (course 12, 18, and 20) and at least two courses in Analytics (course 1, 8, 10, 11, 14, 15).
$$
\sum_{i = 1}^2 x_{i,12} + x_{i,18} + x_{i,20} \geq 2
$$

$$
\sum_{i = 1}^2 x_{i,1} + x_{i,8} + x_{i,10} + x_{i,11} + x_{i,14} + x_{i,15} \geq 2
$$


In [8]:
model.st( x <= a )
model.st( x.sum( axis = 0) <= 1 )
model.st( x[:,5].sum() <= (x[:,9] + x[:,12]).sum() )
## prerequisites
model.st( x[:,7].sum() <= x[:,0].sum() )
model.st( x[:,8].sum() <= x[:,12].sum() )
model.st( x[:,9].sum() <= x[:,0].sum() )
model.st( x[:,10].sum() <= (x[:,0] + x[:,9]).sum() )
model.st( x[:,11].sum() <= x[:,12].sum() )
model.st( x[:,13].sum() <= x[:,3].sum() )
model.st( x[:,14].sum() <= x[:,1].sum() )
model.st( x[:,17].sum() <= x[:,19].sum() )
model.st( x[:,18].sum() <= x[:,19].sum() )

model.st( x[i,:].sum() <= 5 for i in range(2) )

model.st( x[0,0] + x[0,2] + x[0,3] + x[0,4] + x[0,19] >= 3  )

model.st( x[:,11].sum() + x[:,17].sum() <= 1)

model.st( x[:,11].sum() + x[:,17].sum() + x[:,19].sum() >= 2 )

model.st( x[:,0].sum() + x[:,7].sum() + x[:,9].sum() + x[:,10].sum() + x[:,13].sum() + x[:,14].sum() >= 2 )

In [9]:
model.solve( grb )

Being solved by Gurobi...
Solution status: 2
Running time: 0.0125s


In [10]:
print( "--------------------------------------------------------------" )
print( "For the first semester, we will suggest Linda to select course" )
for j in range(n):
    if x.get()[0][j] == 1:
        print(course["Course Code"][j], course["Course Title"][j])

print( "---------------------------------------------------------------" )
print( "For the second semester, we will suggest Linda to select course" )
for j in range(n):
    if x.get()[1][j] == 1:
        print(course["Course Code"][j], course["Course Title"][j])


--------------------------------------------------------------
For the first semester, we will suggest Linda to select course
AB0403 Decision Making with Programming & Analytics
BE1401 Business Operations & Process 
AB0502 Managing Sustainability
BM2501 Market Behaviour
AB1501 Marketing
---------------------------------------------------------------
For the second semester, we will suggest Linda to select course
BA2201 Actuarial Economics 
AB5103 Audit Analytics
AB1202 Statistics & Analysis
BC2410 Prescriptive Analytics: From Data to Decision
BU5644 Smarter Brain? The Science of Decision Making Management
